# Completion API + Context

In [5]:
import os
import dotenv
import pymupdf
from openai import OpenAI
from decimal import Decimal

In [2]:
dotenv.load_dotenv(".env")

GPT_MODEL = "gpt-4o-mini"
DOCUMENTS_FOLDER = os.getcwd() + "/data/raw" # the folder wit rqw PDFs

In [3]:
client = OpenAI()

## Let's collect all the knowledge base into one string

In [19]:
docs_string = ""

for filename in os.listdir(DOCUMENTS_FOLDER):
    print("-- filename", filename)
    
    text = ""
    with pymupdf.open(os.path.join(DOCUMENTS_FOLDER, filename)) as doc:
        for page in doc: # iterate the document pages
            text += page.get_text()
            
    docs_string += f"Document: {filename} \n\n {text} \n\n"
    
f"result string len is: {len(docs_string)} symbols"

-- filename Certification & Education reimbursement policy.pdf
-- filename Time tracking People Force .pdf
-- filename Onboarding Presentation 2024.pdf
-- filename Sunflower game.pdf
-- filename Vacation policy.pdf
-- filename Hardware provision policy.pdf


'result string len is: 27774 symbols'

In [9]:
docs_string[:200]

"Document: Certification & Education reimbursement policy.pdf \n\n Certification & Education Reimbursement Policy\nAt Sloboda Studio, we're all about helping you grow professionally. That's why we've set\n"

## Provide additional context to the bot

In [11]:
system_role = "You answer questions about Sloboda-studio internal rules"
additional_context = "Here is a document base to get information from: \n\n" + docs_string
question = "What is the best way to get highest score in sunflower game?"

Every request have to contain all the context of the conversation. In this case, we have to include the system role and the question.

In [12]:
response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': system_role},
        {'role': 'user', 'content': additional_context},
        {'role': 'user', 'content': question},
    ],
    model=GPT_MODEL,
    temperature=0,
)
# response

In [14]:
# The text of the response
response.choices[0].message.content

'To achieve the highest score in the Sunflower Game at Sloboda Studio, you can focus on the following strategies:\n\n1. **Participate Actively**: Engage in various activities that earn you Sunflower Points. This includes:\n   - Proposing new processes or tools for company improvement (10 points).\n   - Suggesting enhancements for projects (10 points).\n   - Earning points for outstanding work or positive client interactions (15 points).\n   - Offering support to colleagues or enhancing the work environment (10 points).\n   - Taking on mentorship roles (10 points).\n   - Referring candidates to join the team (20 points).\n   - Assisting the marketing team with technical expertise (20 points).\n   - Representing the company at conferences or events (50 points).\n   - Writing articles for the company’s Discourse platform (10 points).\n   - Engaging in social media activities (15 points).\n   - Upselling or expanding project scopes (20 points).\n\n2. **Focus on High-Value Activities**: Pri

## Let's check the usage of the API

In [15]:
response.usage

CompletionUsage(completion_tokens=341, prompt_tokens=7446, total_tokens=7787, prompt_tokens_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0})

In [16]:
pricing = {"input": 0.6 / 1000000, "output": 0.3 / 1000000}
usage = f"Cost: ${Decimal(response.usage.prompt_tokens * pricing['input']) + Decimal(response.usage.completion_tokens * pricing['output'])}"
usage

'Cost: $0.004569899999999999515905985126'